In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import sys

In [26]:
df = pd.read_csv("BigOil.csv", names = ['SKU', 'D', 'h', 'A', 'Fill_Rate','Gr', 'SigmaLT'])
df['Gr'] = df['Gr'].astype(float)
df['Br'] = ((2*df['A']*df['D']*12)/df['h'])**.5 * (1 - df['Fill_Rate']/100)
df['Lz'] = df['Br']/df['SigmaLT']
df['Lz'] = np.around(df['Lz'], decimals=2)
df['Optimal Q w/Br'] = ""
df = df.drop(df.index[len(df)-1])
df = df.round({'Lz':2})
df['Lz'] = (df['Lz']*50).round()/50

In [31]:
df2 = pd.read_csv("Book1.csv", 
                 names = ['z', 'phiz', 'PHIz', 'Lz'], header = 0)
df2['Lz'] = (df2['Lz']*50).round()/50
df2
df2 = df2.dropna()
df3 = df.merge(df2, on='Lz', how='left')

In [32]:
for index, row in df.iterrows():
    Qstar = ((2*row['A']*row['D']*12)/row['h'])**.5
    Br = (Qstar * (1-row['Fill_Rate']/100))
    OptimalQ = (Br/(1-row['Gr'])) + (((Qstar) + (Br/(1-row['Gr']))**2)**.5)
    oldEOQ = Qstar
    while abs(OptimalQ - oldEOQ) >= .01:
        oldEOQ = OptimalQ
        Br = (OptimalQ) * (1-row['Fill_Rate']/100)
        #Lz = Br/row['SigmaLT']
        #Using this new Lz, find the corresponding z-score and G(r*) value
        OptimalQ = (Br/(1-row['Gr'])) + (((2*row['A']*oldEOQ)/row['h']) + (Br/(1-row['Gr']))**2)**.5
    df['Optimal Q w/Br'][index] = OptimalQ
    #print(int(row['SKU']), OptimalQ)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
df

,SKU,D,h,A,Fill_Rate,Gr,SigmaLT,Br,Lz,Optimal Q w/Br
0,93494.0,473.9848,3.566,200.0,80.0,0.00307,58.114277,159.750473,2.76,187.35
1,930403.0,163.3333,2.378,200.0,80.0,0.13136,93.148627,114.837058,1.24,311.767
2,930523.0,110.0000,2.378,200.0,90.0,0.38209,88.994382,47.120624,0.52,248.698
3,930616.0,293.3333,2.378,200.0,90.0,0.29460,107.129205,76.947653,0.72,234.761
4,930620.0,234.7500,2.378,200.0,90.0,0.19489,74.576166,68.836296,0.92,223.793
5,930625.0,36.6667,2.378,200.0,90.0,0.29460,38.297084,27.205118,0.72,234.759
6,930630.0,220.4167,2.378,200.0,80.0,0.06426,85.447796,133.403412,1.56,293.787
7,930631.0,219.7748,2.242,200.0,80.0,0.05938,83.672103,137.189779,1.64,310.406
8,931298.0,176.6739,2.378,200.0,80.0,0.14007,105.733076,119.434800,1.12,314.483
9,931300.0,32.0000,2.378,200.0,90.0,0.14457,24.408642,25.414975,1.04,219.527
